In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from laughing import phi15

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-28 06:03:48.969310: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 06:03:49.991280: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib:/o

In [3]:
model, _, tokenizer = phi15.load_model_and_tokenizer()

loading model...
loading tokenizer...


In [4]:
import json
# this file contains original instruction and test cases generated by fine-tuned phi1_5 and gpt4 teacher
test_cases_file = '../data/phi15_generated_test_cases_train.json'

with open(test_cases_file, encoding='utf-8') as fin:
  data = json.load(fin)

In [8]:
len(data)

3297

In [9]:
data[0]

{'instruction': 'Fix the following function to make it comply with PEP8 standards.',
 'input': 'def f(x):\nk= x+4\nreturn k',
 'output': 'def f(x):\n    k = x + 4\n    return k',
 'test_cases': 'def test_positive_number():\n    assert f(5) == 9\n\ndef test_negative_number():\n    assert f(-3) == 1\n\ndef test_zero():\n    assert f(0) == 4\n\ndef test_float_number():\n    assert f(2.5) == 6.5\n',
 'phi_finetuned_test_cases': "Generate unit tests for the below problem and its solution in Python.\n    Write each unit test as a seperate Python function with meaningful name that starts with 'test_'.\n\n    Problem:\n    Fix the following function to make it comply with PEP8 standards.\n\n    Solution:\n    def f(x):\n    k = x + 4\n    return k\n\n    Test cases:\n    def test_generate_sum_of_squares():\n        assert f(5) == 25\n        assert f(6) == 41\n        assert f(7) == 100\n        assert f(8) == 121\n        assert f(9) == 121\n        assert f(10) == 121\n        assert f(11) =

In [5]:
# generate outputs for the entire dataset
batch_size = 20
responses = []

def generate(model, tokenizer, data, responses=[]):
    for i in range(len(responses), len(data), batch_size):
        print(i, end=' ')
        prompts = phi15.make_prompts_for_code_gen(data[i:i+batch_size])
        responses.extend(phi15.gen_n_prompts(model, tokenizer, prompts))
    return responses


In [6]:
responses = generate(model, tokenizer, data[0:5], responses)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 

In [64]:
for task, response in zip(data, responses):
    task['phi_base_output'] = phi15.extract_program(response)
    #task['gpt4_test_cases'] = task['test_cases']
    #task.pop('test_cases')

In [68]:
# add the program output from base phi15 model
with open('../data/code_alpaca_v2.json', encoding='utf-8', mode='w') as fout:
    json.dump(data, fout)

In [ ]:
# load phi15 fine-tuned model and generate test cases using the generated program
# run batch eval for test cases
# prepare training for RRTH
# train model


In [2]:
from peft import PeftModel, PeftConfig, AutoPeftModelForCausalLM

#model, peft_model, tokenizer = phi15.load_model_and_tokenizer(lora_rank=2)
#peft_config = PeftConfig.from_pretrained('../../phi15_finetuned_for_tests_generation/')
#peft_model = PeftModel.from_pretrained(model, '../../phi15_finetuned_for_tests_generation/')
peft_model = AutoPeftModelForCausalLM.from_pretrained(
    '../phi15_finetuned_outputs_20230928/checkpoint-824/', trust_remote_code=True)
#model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True, torch_dtype=torch.float32)
#peft_model = PeftModel.from_pretrained(model, "Vasanth/phi-1_5-finetuned-gsm8k", from_transformers=True)
#model = peft_model.merge_and_unload()


In [3]:
for n, p in peft_model.named_parameters():
    if 'lora' in n:
        print(n)
        print(p)

base_model.model.layers.1.mixer.Wqkv.lora_A.default.weight
Parameter containing:
tensor([[ 0.0034,  0.0221,  0.0197,  ..., -0.0241, -0.0305, -0.0147],
        [ 0.0086,  0.0162,  0.0369,  ..., -0.0409, -0.0021, -0.0226]])
base_model.model.layers.1.mixer.Wqkv.lora_B.default.weight
Parameter containing:
tensor([[-0.0027, -0.0005],
        [-0.0052, -0.0022],
        [-0.0059, -0.0006],
        ...,
        [-0.0026, -0.0043],
        [-0.0062, -0.0070],
        [-0.0049, -0.0048]])
base_model.model.layers.2.mixer.Wqkv.lora_A.default.weight
Parameter containing:
tensor([[ 0.0139,  0.0159,  0.0140,  ...,  0.0372,  0.0312, -0.0363],
        [ 0.0380,  0.0154,  0.0057,  ...,  0.0067,  0.0117, -0.0042]])
base_model.model.layers.2.mixer.Wqkv.lora_B.default.weight
Parameter containing:
tensor([[-0.0102, -0.0101],
        [ 0.0158,  0.0164],
        [ 0.0144,  0.0154],
        ...,
        [-0.0041, -0.0041],
        [-0.0084, -0.0091],
        [ 0.0051,  0.0055]])
base_model.model.layers.3.mixe

In [11]:
import json

with open('../data/code_alpaca_v2.json', encoding='utf-8') as fin:
    data = json.load(fin)

In [12]:
# I can create this but it won't help me map to original index ... 
new_tasks = [{'instruction': t['instruction'], 'output': t['output']}
             for t in data if t['phi_base_output'] != '']

In [13]:
prompts = phi15.make_prompts_for_tests_gen(new_tasks[:5])

In [14]:
print(prompts[0])


        Problem:
        Fix the following function to make it comply with PEP8 standards.

        Solution:
        def f(x):
    k = x + 4
    return k

        Test cases:
        


In [20]:
batch_size = 5

def generate(model, tokenizer, data, responses=[]):
    for i in range(len(responses), len(data), batch_size):
        print(i, end=' ')
        #prompts = phi15.make_prompts_for_tests_gen(data[i:i+batch_size])
        responses.extend(phi15.gen_n_prompts(model, tokenizer, data))
    return responses


In [21]:
from laughing import utils

utils.free_gpu_memory()

Allocated GPU Memory: 7.94 GB
Maximum Allocated GPU Memory: 11.88 GB
Available GPU Memory: 7.85 GB


In [23]:
responses = generate(peft_model, tokenizer, prompts, [])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 

In [24]:
for x in responses:
    print(x)
    print('\n======================')

         Problem:          Fix the following function to make it comply with PEP8 standards.          Solution:          def f(x):      k = x + 4      return k          Test cases:          def f(x):
    return x + 4

def g(x):
    return x + 4

Problem:

def g(x):
    return x + 4

    def g(x):
      return x + 4

    def test_g(x):
      return x + 4

    def test_g_no_return(x):
      return x + 4

    def test_g_with_return(x):
      return x + 4

    def test_g_with_return(x):
      return x + 4

    def test_g_with_return_no_return(x):
      return x + 4

    def test_g_with_return_no_return(x):
      return x + 4

    def test_g_with_return_no_return_no_return(x):
      return x + 4

    def test_g_with_return_no_return_no_return(x):
      return x + 4

    def test_g_with_return_no_return_no_return(x):
      return x + 4

    def test_g_with_return_no_return_no_return(x):
      return x + 4

    def test_g_with_return_no_return_no_return(x):
      return x + 4

    def test_g_

In [142]:
_ = peft_model.eval()

In [148]:
import torch

@torch.inference_mode()
def gen_test_cases(model, tokenizer, prompts, max_new_tokens:int=300):
    """Generate test cases for a batch of tasks using a model."""
    inputs = tokenizer(prompts, return_tensors="pt", return_attention_mask=False,
                       padding=True, truncation=True).to('cuda')
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,
                             eos_token_id=tokenizer.eos_token_id)
    texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return texts



In [149]:
responses = gen_test_cases(peft_model, tokenizer, prompts)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [150]:
for x in responses:
    print(x)
    print('\n=====================\n')

Generate unit tests for the below problem and its solution in Python.          Write each unit test as a seperate Python function with meaningful name that starts with 'test_'.          Problem:          Fix the following function to make it comply with PEP8 standards.          Solution:          def f(x):      k = x + 4      return k          Test cases:          def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    return x + 4    Test cases:    def f(x):    retu